Consolidated Scripts for Three Models



1. VGG16 Transfer Learning



In [ ]:
# Install required packages
# !pip install numpy==1.26.4 tensorflow==2.15.0 pandas scikit-learn matplotlib

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import os
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils import class_weight
import pandas as pd
import warnings
from PIL import Image

# Suppress warnings
warnings.filterwarnings('ignore')

# Set random seed
tf.random.set_seed(42)
np.random.seed(42)

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define hyperparameters
data_dir = '/content/drive/MyDrive/data'
image_height = 224
image_width = 224
batch_size = 32
n_folds = 5

# Load dataset
image_paths = []
labels = []
print(f"\nScanning directory: {data_dir}")
for filename in os.listdir(data_dir):
    if filename.lower().endswith(('.jpeg', '.jpg', '.png')):
        file_path = os.path.join(data_dir, filename)
        try:
            with Image.open(file_path) as img:
                img.verify()
            image_paths.append(file_path)
            label = 1 if filename.lower().startswith('y') else 0
            labels.append(label)
        except Exception as e:
            print(f"Skipping corrupt file: {filename} ({str(e)})")
image_paths = np.array(image_paths)
labels = np.array(labels)

if len(image_paths) == 0:
    raise ValueError(f"No valid images found in {data_dir}.")
print(f"\nLoaded {len(image_paths)} images: {sum(labels)} truffle cracks, {len(labels) - sum(labels)} other cracks")

# Split dataset into train+val (240) and test (60)
train_val_paths, test_paths, train_val_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42
)

# Compute class weights
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
print(f"Class weights: {class_weight_dict}")

# Define data generators
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3]
)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Cross-validation
fold_accuracies = []
fold_precisions = []
fold_recalls = []
fold_f1_scores = []
fold_aucs = []
fold_histories = []

skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
for fold, (train_idx, val_idx) in enumerate(skf.split(train_val_paths, train_val_labels)):
    print(f"\nTraining Fold {fold + 1}/{n_folds}")
    train_paths, val_paths = train_val_paths[train_idx], train_val_paths[val_idx]
    train_labels, val_labels = train_val_labels[train_idx], train_val_labels[val_idx]

    train_generator = train_datagen.flow_from_dataframe(
        dataframe=pd.DataFrame({'filename': train_paths, 'class': train_labels.astype(str)}),
        x_col='filename', y_col='class', target_size=(image_height, image_width),
        batch_size=batch_size, class_mode='binary', shuffle=True
    )
    val_generator = test_datagen.flow_from_dataframe(
        dataframe=pd.DataFrame({'filename': val_paths, 'class': val_labels.astype(str)}),
        x_col='filename', y_col='class', target_size=(image_height, image_width),
        batch_size=batch_size, class_mode='binary', shuffle=False
    )

    # Build VGG16 model
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_height, image_width, 3))
    base_model.trainable = False
    model = models.Sequential([
        base_model,
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    # Compile model
    lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

    # Train model
    history = model.fit(
        train_generator, epochs=50, validation_data=val_generator,
        callbacks=[early_stopping, lr_schedule], class_weight=class_weight_dict, verbose=1
    )
    fold_histories.append(history.history)

    # Evaluate on validation set
    val_predictions = model.predict(val_generator, verbose=0)
    val_predictions_binary = (val_predictions > 0.5).astype(int).flatten()
    val_true = val_labels
    fold_accuracies.append(accuracy_score(val_true, val_predictions_binary))
    fold_precisions.append(precision_score(val_true, val_predictions_binary, zero_division=0))
    fold_recalls.append(recall_score(val_true, val_predictions_binary, zero_division=0))
    fold_f1_scores.append(f1_score(val_true, val_predictions_binary, zero_division=0))
    fold_aucs.append(roc_auc_score(val_true, val_predictions))
    print(f"Fold {fold + 1} - Accuracy: {fold_accuracies[-1]:.3f}, Precision: {fold_precisions[-1]:.3f}, "
          f"Recall: {fold_recalls[-1]:.3f}, F1 Score: {fold_f1_scores[-1]:.3f}, AUC: {fold_aucs[-1]:.3f}")

# Print cross-validation results
print("\nCross-Validation Results (Mean ± Std):")
print(f"Accuracy: {np.mean(fold_accuracies):.3f} ± {np.std(fold_accuracies):.3f}")
print(f"Precision: {np.mean(fold_precisions):.3f} ± {np.std(fold_precisions):.3f}")
print(f"Recall: {np.mean(fold_recalls):.3f} ± {np.std(fold_recalls):.3f}")
print(f"F1 Score: {np.mean(fold_f1_scores):.3f} ± {np.std(fold_f1_scores):.3f}")
print(f"AUC: {np.mean(fold_aucs):.3f} ± {np.std(fold_aucs):.3f}")

# Train final model on full train+val data
train_generator = train_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': train_val_paths, 'class': train_val_labels.astype(str)}),
    x_col='filename', y_col='class', target_size=(image_height, image_width),
    batch_size=batch_size, class_mode='binary', shuffle=True
)
model = models.Sequential([
    VGG16(weights='imagenet', include_top=False, input_shape=(image_height, image_width, 3)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])
model.layers[0].trainable = False
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])
model.fit(train_generator, epochs=50, callbacks=[early_stopping, lr_schedule], class_weight=class_weight_dict, verbose=1)
model.save('/content/drive/MyDrive/truffle_classification_model_vgg16.keras')

# Test set evaluation
test_generator = test_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': test_paths, 'class': test_labels.astype(str)}),
    x_col='filename', y_col='class', target_size=(image_height, image_width),
    batch_size=batch_size, class_mode='binary', shuffle=False
)
test_predictions = model.predict(test_generator, verbose=1)
test_predictions_binary = (test_predictions > 0.5).astype(int).flatten()
test_true = test_generator.classes

# Calculate test metrics
test_accuracy = accuracy_score(test_true, test_predictions_binary)
test_precision = precision_score(test_true, test_predictions_binary, zero_division=0)
test_recall = recall_score(test_true, test_predictions_binary, zero_division=0)
test_f1 = f1_score(test_true, test_predictions_binary, zero_division=0)
test_auc = roc_auc_score(test_true, test_predictions)
print(f"\nVGG16 Test Results:")
print(f"Test Accuracy: {test_accuracy:.3f}")
print(f"Test Precision: {test_precision:.3f}")
print(f"Test Recall: {test_recall:.3f}")
print(f"Test F1 Score: {test_f1:.3f}")
print(f"Test AUC: {test_auc:.3f}")

# Optimize recall
threshold = 0.3
test_predictions_binary_opt = (test_predictions > threshold).astype(int).flatten()
test_recall_opt = recall_score(test_true, test_predictions_binary_opt, zero_division=0)
print(f"Test Recall (threshold={threshold}): {test_recall_opt:.3f}")

# Generate figures
fpr, tpr, _ = roc_curve(test_true, test_predictions)
plt.figure()
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {test_auc:.3f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('VGG16 ROC Curve (Test Set)')
plt.legend(loc='lower right')
plt.savefig('/content/drive/MyDrive/roc_curve_vgg16.png')
plt.close()

cm = confusion_matrix(test_true, test_predictions_binary)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Other Cracks', 'Truffle Cracks'])
disp.plot(cmap='Blues')
plt.title('VGG16 Confusion Matrix (Test Set)')
plt.savefig('/content/drive/MyDrive/confusion_matrix_vgg16.png')
plt.close()

min_epochs = min(len(h['accuracy']) for h in fold_histories)
avg_train_acc = np.mean([h['accuracy'][:min_epochs] for h in fold_histories], axis=0)
avg_val_acc = np.mean([h['val_accuracy'][:min_epochs] for h in fold_histories], axis=0)
avg_train_loss = np.mean([h['loss'][:min_epochs] for h in fold_histories], axis=0)
avg_val_loss = np.mean([h['val_loss'][:min_epochs] for h in fold_histories], axis=0)
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(avg_train_acc, label='Training Accuracy')
plt.plot(avg_val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title(f'VGG16 Average Accuracy Across Folds')
plt.subplot(1, 2, 2)
plt.plot(avg_train_loss, label='Training Loss')
plt.plot(avg_val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title(f'VGG16 Average Loss Across Folds')
plt.tight_layout()
plt.savefig('/content/drive/MyDrive/training_plots_vgg16.png')
plt.close()

Mounted at /content/drive

Scanning directory: /content/drive/MyDrive/data

Loaded 300 images: 150 truffle cracks, 150 other cracks
Class weights: {0: np.float64(1.0), 1: np.float64(1.0)}

Training Fold 1/5
Found 192 validated image filenames belonging to 2 classes.
Found 48 validated image filenames belonging to 2 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 19s/step - accuracy: 0.4622 - auc: 0.4762 - loss: 5.2279 

2. SVM with MobileNetV2 Features



In [ ]:
# Install required packages
# !pip install numpy==1.26.4 tensorflow==2.15.0 pandas scikit-learn matplotlib joblib

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import os
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix, ConfusionMatrixDisplay
from sklearn.svm import SVC
from sklearn.utils import class_weight
import pandas as pd
from PIL import Image
from joblib import dump

# Suppress warnings
warnings.filterwarnings('ignore')

# Set random seed
tf.random.set_seed(42)
np.random.seed(42)

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define hyperparameters
data_dir = '/content/drive/MyDrive/data'
image_height = 224
image_width = 224
batch_size = 32
n_folds = 5

# Load dataset
image_paths = []
labels = []
print(f"\nScanning directory: {data_dir}")
for filename in os.listdir(data_dir):
    if filename.lower().endswith(('.jpeg', '.jpg', '.png')):
        file_path = os.path.join(data_dir, filename)
        try:
            with Image.open(file_path) as img:
                img.verify()
            image_paths.append(file_path)
            label = 1 if filename.lower().startswith('y') else 0
            labels.append(label)
        except Exception as e:
            print(f"Skipping corrupt file: {filename} ({str(e)})")
image_paths = np.array(image_paths)
labels = np.array(labels)

if len(image_paths) == 0:
    raise ValueError(f"No valid images found in {data_dir}.")
print(f"\nLoaded {len(image_paths)} images: {sum(labels)} truffle cracks, {len(labels) - sum(labels)} other cracks")

# Split dataset
train_val_paths, test_paths, train_val_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42
)

# Compute class weights
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
print(f"Class weights: {class_weight_dict}")

# Define data generators
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Load MobileNetV2 for feature extraction
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(image_height, image_width, 3))
base_model.trainable = False
feature_model = tf.keras.Sequential([base_model, tf.keras.layers.GlobalAveragePooling2D()])

def extract_features(generator):
    features = feature_model.predict(generator, verbose=1)
    labels = generator.classes
    return features, labels

# Cross-validation
fold_accuracies = []
fold_precisions = []
fold_recalls = []
fold_f1_scores = []
fold_aucs = []

skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
for fold, (train_idx, val_idx) in enumerate(skf.split(train_val_paths, train_val_labels)):
    print(f"\nTraining Fold {fold + 1}/{n_folds}")
    train_paths, val_paths = train_val_paths[train_idx], train_val_paths[val_idx]
    train_labels, val_labels = train_val_labels[train_idx], train_val_labels[val_idx]

    train_generator = datagen.flow_from_dataframe(
        dataframe=pd.DataFrame({'filename': train_paths, 'class': train_labels.astype(str)}),
        x_col='filename', y_col='class', target_size=(image_height, image_width),
        batch_size=batch_size, class_mode='binary', shuffle=False
    )
    val_generator = datagen.flow_from_dataframe(
        dataframe=pd.DataFrame({'filename': val_paths, 'class': val_labels.astype(str)}),
        x_col='filename', y_col='class', target_size=(image_height, image_width),
        batch_size=batch_size, class_mode='binary', shuffle=False
    )

    train_features, train_labels = extract_features(train_generator)
    val_features, val_labels = extract_features(val_generator)

    svm = SVC(kernel='linear', probability=True, class_weight=class_weight_dict, random_state=42)
    svm.fit(train_features, train_labels)

    val_preds = svm.predict_proba(val_features)[:, 1]
    val_preds_binary = (val_preds > 0.5).astype(int)
    fold_accuracies.append(accuracy_score(val_labels, val_preds_binary))
    fold_precisions.append(precision_score(val_labels, val_preds_binary, zero_division=0))
    fold_recalls.append(recall_score(val_labels, val_preds_binary, zero_division=0))
    fold_f1_scores.append(f1_score(val_labels, val_preds_binary, zero_division=0))
    fold_aucs.append(roc_auc_score(val_labels, val_preds))
    print(f"Fold {fold + 1} - Accuracy: {fold_accuracies[-1]:.3f}, Precision: {fold_precisions[-1]:.3f}, "
          f"Recall: {fold_recalls[-1]:.3f}, F1 Score: {fold_f1_scores[-1]:.3f}, AUC: {fold_aucs[-1]:.3f}")

# Print cross-validation results
print("\nCross-Validation Results (Mean ± Std):")
print(f"Accuracy: {np.mean(fold_accuracies):.3f} ± {np.std(fold_accuracies):.3f}")
print(f"Precision: {np.mean(fold_precisions):.3f} ± {np.std(fold_precisions):.3f}")
print(f"Recall: {np.mean(fold_recalls):.3f} ± {np.std(fold_recalls):.3f}")
print(f"F1 Score: {np.mean(fold_f1_scores):.3f} ± {np.std(fold_f1_scores):.3f}")
print(f"AUC: {np.mean(fold_aucs):.3f} ± {np.std(fold_aucs):.3f}")

# Train final SVM on full train+val data
train_generator = datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': train_val_paths, 'class': train_val_labels.astype(str)}),
    x_col='filename', y_col='class', target_size=(image_height, image_width),
    batch_size=batch_size, class_mode='binary', shuffle=False
)
train_features, train_labels = extract_features(train_generator)
svm = SVC(kernel='linear', probability=True, class_weight=class_weight_dict, random_state=42)
svm.fit(train_features, train_labels)
dump(svm, '/content/drive/MyDrive/svm_model.joblib')

# Test set evaluation
test_generator = datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': test_paths, 'class': test_labels.astype(str)}),
    x_col='filename', y_col='class', target_size=(image_height, image_width),
    batch_size=batch_size, class_mode='binary', shuffle=False
)
test_features, test_labels = extract_features(test_generator)
test_preds = svm.predict_proba(test_features)[:, 1]
test_preds_binary = (test_preds > 0.5).astype(int)
decision_scores = svm.decision_function(test_features)

# Calculate test metrics
test_accuracy = accuracy_score(test_labels, test_preds_binary)
test_precision = precision_score(test_labels, test_preds_binary, zero_division=0)
test_recall = recall_score(test_labels, test_preds_binary, zero_division=0)
test_f1 = f1_score(test_labels, test_preds_binary, zero_division=0)
test_auc = roc_auc_score(test_labels, test_preds)
print(f"\nSVM Test Results (default threshold=0.5):")
print(f"Test Accuracy: {test_accuracy:.3f}")
print(f"Test Precision: {test_precision:.3f}")
print(f"Test Recall: {test_recall:.3f}")
print(f"Test F1 Score: {test_f1:.3f}")
print(f"Test AUC: {test_auc:.3f}")

# Optimize recall at threshold=-0.3
threshold = -0.3
test_preds_binary_opt = (decision_scores > threshold).astype(int)
test_accuracy_opt = accuracy_score(test_labels, test_preds_binary_opt)
test_precision_opt = precision_score(test_labels, test_preds_binary_opt, zero_division=0)
test_recall_opt = recall_score(test_labels, test_preds_binary_opt, zero_division=0)
test_f1_opt = f1_score(test_labels, test_preds_binary_opt, zero_division=0)
print(f"\nSVM Test Results (threshold=-0.3):")
print(f"Test Accuracy: {test_accuracy_opt:.3f}")
print(f"Test Precision: {test_precision_opt:.3f}")
print(f"Test Recall: {test_recall_opt:.3f}")
print(f"Test F1 Score: {test_f1_opt:.3f}")
print(f"Test AUC: {test_auc:.3f}")

# Generate figures
fpr, tpr, _ = roc_curve(test_labels, test_preds)
plt.figure()
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {test_auc:.3f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('SVM ROC Curve (Test Set)')
plt.legend(loc='lower right')
plt.savefig('/content/drive/MyDrive/roc_curve_svm.png')
plt.close()

cm = confusion_matrix(test_labels, test_preds_binary_opt)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Other Cracks', 'Truffle Cracks'])
disp.plot(cmap='Blues')
plt.title('SVM Confusion Matrix (Test Set, threshold=-0.3)')
plt.savefig('/content/drive/MyDrive/confusion_matrix_svm.png')
plt.close()

3. EfficientNetV2B0



In [ ]:
# Install required packages
# !pip install numpy==1.26.4 tensorflow==2.15.0 pandas scikit-learn matplotlib tqdm

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
import os
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils import class_weight
import pandas as pd
from PIL import Image

# Suppress warnings
warnings.filterwarnings('ignore')

# Set random seed
tf.random.set_seed(42)
np.random.seed(42)

# Enable mixed precision
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define hyperparameters
data_dir = '/content/drive/MyDrive/data'
image_height = 224
image_width = 224
batch_size = 16
n_folds = 5
initial_learning_rate = 3e-4

# Load dataset
image_paths = []
labels = []
print(f"\nScanning directory: {data_dir}")
for filename in os.listdir(data_dir):
    if filename.lower().endswith(('.jpeg', '.jpg', '.png')):
        file_path = os.path.join(data_dir, filename)
        try:
            with Image.open(file_path) as img:
                img.verify()
            image_paths.append(file_path)
            label = 1 if filename.lower().startswith('y') else 0
            labels.append(label)
        except Exception as e:
            print(f"Skipping corrupt file: {filename} ({str(e)})")
image_paths = np.array(image_paths)
labels = np.array(labels)

if len(image_paths) == 0:
    raise ValueError(f"No valid images found in {data_dir}.")
print(f"\nLoaded {len(image_paths)} images: {sum(labels)} truffle cracks, {len(labels) - sum(labels)} other cracks")

# Split dataset
train_val_paths, test_paths, train_val_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42
)

# Compute class weights
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
print(f"Class weights: {class_weight_dict}")

# Define data generators
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2]
)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
tta_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=5,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.05,
    horizontal_flip=True
)

# Cross-validation
fold_accuracies = []
fold_precisions = []
fold_recalls = []
fold_f1_scores = []
fold_aucs = []
fold_histories = []

skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
for fold, (train_idx, val_idx) in enumerate(skf.split(train_val_paths, train_val_labels)):
    print(f"\nTraining Fold {fold + 1}/{n_folds}")
    train_paths, val_paths = train_val_paths[train_idx], train_val_paths[val_idx]
    train_labels, val_labels = train_val_labels[train_idx], train_val_labels[val_idx]

    train_generator = train_datagen.flow_from_dataframe(
        dataframe=pd.DataFrame({'filename': train_paths, 'class': train_labels.astype(str)}),
        x_col='filename', y_col='class', target_size=(image_height, image_width),
        batch_size=batch_size, class_mode='binary', shuffle=True
    )
    val_generator = test_datagen.flow_from_dataframe(
        dataframe=pd.DataFrame({'filename': val_paths, 'class': val_labels.astype(str)}),
        x_col='filename', y_col='class', target_size=(image_height, image_width),
        batch_size=batch_size, class_mode='binary', shuffle=False
    )
    val_tta_generator = tta_datagen.flow_from_dataframe(
        dataframe=pd.DataFrame({'filename': val_paths, 'class': val_labels.astype(str)}),
        x_col='filename', y_col='class', target_size=(image_height, image_width),
        batch_size=batch_size, class_mode='binary', shuffle=False
    )

    # Build EfficientNetV2B0 model
    base_model = EfficientNetV2B0(weights='imagenet', include_top=False, input_shape=(image_height, image_width, 3))
    base_model.trainable = False
    inputs = tf.keras.Input(shape=(image_height, image_width, 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = layers.Dropout(0.7)(x)
    outputs = layers.Dense(1, activation='sigmoid', dtype='float32')(x)
    model = Model(inputs, outputs)

    lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=8, min_lr=1e-6)
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=initial_learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

    history = model.fit(
        train_generator, epochs=50, validation_data=val_generator,
        callbacks=[early_stopping, lr_schedule], class_weight=class_weight_dict, verbose=1
    )
    fold_histories.append(history.history)

    # Evaluate with TTA
    val_predictions = np.mean([model.predict(val_tta_generator, verbose=0) for _ in range(5)], axis=0)
    val_predictions_binary = (val_predictions > 0.5).astype(int).flatten()
    val_true = val_labels
    fold_accuracies.append(accuracy_score(val_true, val_predictions_binary))
    fold_precisions.append(precision_score(val_true, val_predictions_binary, zero_division=0))
    fold_recalls.append(recall_score(val_true, val_predictions_binary, zero_division=0))
    fold_f1_scores.append(f1_score(val_true, val_predictions_binary, zero_division=0))
    fold_aucs.append(roc_auc_score(val_true, val_predictions))
    print(f"Fold {fold + 1} - Accuracy: {fold_accuracies[-1]:.3f}, Precision: {fold_precisions[-1]:.3f}, "
          f"Recall: {fold_recalls[-1]:.3f}, F1 Score: {fold_f1_scores[-1]:.3f}, AUC: {fold_aucs[-1]:.3f}")

# Print cross-validation results
print("\nCross-Validation Results (Mean ± Std):")
print(f"Accuracy: {np.mean(fold_accuracies):.3f} ± {np.std(fold_accuracies):.3f}")
print(f"Precision: {np.mean(fold_precisions):.3f} ± {np.std(fold_precisions):.3f}")
print(f"Recall: {np.mean(fold_recalls):.3f} ± {np.std(fold_recalls):.3f}")
print(f"F1 Score: {np.mean(fold_f1_scores):.3f} ± {np.std(fold_f1_scores):.3f}")
print(f"AUC: {np.mean(fold_aucs):.3f} ± {np.std(fold_aucs):.3f}")

# Train final model on full train+val data
train_generator = train_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': train_val_paths, 'class': train_val_labels.astype(str)}),
    x_col='filename', y_col='class', target_size=(image_height, image_width),
    batch_size=batch_size, class_mode='binary', shuffle=True
)
base_model = EfficientNetV2B0(weights='imagenet', include_top=False, input_shape=(image_height, image_width, 3))
base_model.trainable = False
inputs = tf.keras.Input(shape=(image_height, image_width, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = layers.Dropout(0.7)(x)
outputs = layers.Dense(1, activation='sigmoid', dtype='float32')(x)
model = Model(inputs, outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=initial_learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])
model.fit(train_generator, epochs=50, callbacks=[early_stopping, lr_schedule], class_weight=class_weight_dict, verbose=1)
model.save('/content/drive/MyDrive/truffle_classification_model_effnet.keras')

# Test set evaluation with TTA
test_generator = test_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': test_paths, 'class': test_labels.astype(str)}),
    x_col='filename', y_col='class', target_size=(image_height, image_width),
    batch_size=batch_size, class_mode='binary', shuffle=False
)
test_tta_generator = tta_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': test_paths, 'class': test_labels.astype(str)}),
    x_col='filename', y_col='class', target_size=(image_height, image_width),
    batch_size=batch_size, class_mode='binary', shuffle=False
)
test_predictions = np.mean([model.predict(test_tta_generator, verbose=1) for _ in range(5)], axis=0)
test_predictions_binary = (test_predictions > 0.5).astype(int).flatten()
test_true = test_labels

# Calculate test metrics
test_accuracy = accuracy_score(test_true, test_predictions_binary)
test_precision = precision_score(test_true, test_predictions_binary, zero_division=0)
test_recall = recall_score(test_true, test_predictions_binary, zero_division=0)
test_f1 = f1_score(test_true, test_predictions_binary, zero_division=0)
test_auc = roc_auc_score(test_true, test_predictions)
print(f"\nEfficientNetV2B0 Test Results (with TTA):")
print(f"Test Accuracy: {test_accuracy:.3f}")
print(f"Test Precision: {test_precision:.3f}")
print(f"Test Recall: {test_recall:.3f}")
print(f"Test F1 Score: {test_f1:.3f}")
print(f"Test AUC: {test_auc:.3f}")

# Generate figures
fpr, tpr, _ = roc_curve(test_true, test_predictions)
plt.figure()
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {test_auc:.3f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('EfficientNetV2B0 ROC Curve (Test Set)')
plt.legend(loc='lower right')
plt.savefig('/content/drive/MyDrive/roc_curve_effnet.png')
plt.close()

cm = confusion_matrix(test_true, test_predictions_binary)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Other Cracks', 'Truffle Cracks'])
disp.plot(cmap='Blues')
plt.title('EfficientNetV2B0 Confusion Matrix (Test Set)')
plt.savefig('/content/drive/MyDrive/confusion_matrix_effnet.png')
plt.close()

min_epochs = min(len(h['accuracy']) for h in fold_histories)
avg_train_acc = np.mean([h['accuracy'][:min_epochs] for h in fold_histories], axis=0)
avg_val_acc = np.mean([h['val_accuracy'][:min_epochs] for h in fold_histories], axis=0)
avg_train_loss = np.mean([h['loss'][:min_epochs] for h in fold_histories], axis=0)
avg_val_loss = np.mean([h['val_loss'][:min_epochs] for h in fold_histories], axis=0)
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(avg_train_acc, label='Training Accuracy')
plt.plot(avg_val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title(f'EfficientNetV2B0 Average Accuracy Across Folds')
plt.subplot(1, 2, 2)
plt.plot(avg_train_loss, label='Training Loss')
plt.plot(avg_val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title(f'EfficientNetV2B0 Average Loss Across Folds')
plt.tight_layout()
plt.savefig('/content/drive/MyDrive/training_plots_effnet.png')
plt.close()